# Tracy Giskard Demo

## 1. Launch Tracy simulation

In [1]:
import rospy
import time
from utils import open_desktop, run_command, create_marker, publish_marker_array, TracyDemo
from std_msgs.msg import ColorRGBA
import random

In [2]:
# Open a desktop GUI left sidecar
# Note: if you see connection error, try running this code cell again
open_desktop()

In [3]:
# Run the roslaunch in a background process
# Note: To see the logs, run this command in a terminal tab. 
# Run this code cell will restart the simulation.
run_command('roslaunch giskardpy giskardpy_tracy_standalone.launch')

## 2. Control the Tracy with the Giskard Python Interface

In [4]:
# initialize a ros node and the python wrapper for giskard
import rospy
from pprint import pprint
from giskardpy.python_interface import GiskardWrapper
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped
rospy.init_node('gk_python_demo')
giskard_wrapper = GiskardWrapper()

### List Joints and links

In [ ]:
# List all the controlled joints
robot_joints = giskard_wrapper.get_controlled_joints(giskard_wrapper.get_group_names()[0])
# List all the links
robot_links = giskard_wrapper.get_group_info(giskard_wrapper.get_group_names()[0]).links

print('Joints:')
pprint(robot_joints)
print('Links:')
pprint(robot_links)

### Define a default pose

In [6]:
default_pose = {
        'left_shoulder_pan_joint': 2.539670467376709,
        'left_shoulder_lift_joint': -1.46823854119096,
        'left_elbow_joint': 2.1197431723224085,
        'left_wrist_1_joint': -1.4825000625899811,
        'left_wrist_2_joint': 5.467689037322998,
        'left_wrist_3_joint': -0.9808381239520472,
        'right_shoulder_pan_joint': 3.7588136196136475,
        'right_shoulder_lift_joint': -1.7489210567870082,
        'right_elbow_joint': -2.054229259490967,
        'right_wrist_1_joint': -1.6140786610045375,
        'right_wrist_2_joint': 0.7295855283737183,
        'right_wrist_3_joint': 3.944669485092163,
    }

# teleport giskard into the default pose (only works in simulation)
giskard_wrapper.set_json_goal('SetSeedConfiguration',
                          seed_configuration=default_pose)
giskard_wrapper.allow_all_collisions()
giskard_wrapper.plan_and_execute()

# move to default pose(Not )
def reset_to_default_pose():
    giskard_wrapper.set_joint_goal(default_pose)
    giskard_wrapper.plan_and_execute()

### Spawn and attach cylinders to the hands

In [7]:
cylinder_height = 0.121
def spawn_cylinder(hand='r', color=(1, 1, 0, 1)):
    cylinder_name = f'C_{time.time()}'
    frame_id = f'{hand}_gripper_tool_frame'
    pose = PoseStamped()
    pose.header.frame_id = frame_id
    pose.pose.position.z = cylinder_height / 5
    pose.pose.orientation.w = 1
    giskard_wrapper.add_cylinder(name=cylinder_name,
                             height=cylinder_height,
                             radius=0.0225,
                             pose=pose,
                             parent_link=frame_id)
    # dye it
    giskard_wrapper.dye_group(cylinder_name, color)
    return cylinder_name

### Call a goal to "insert" the cylinder at a point

In [8]:
def insert_cylider_to(x, y, cylinder_name):
    hole_point = PointStamped()
    hole_point.header.frame_id = 'table'
    hole_point.point.x = x
    hole_point.point.y = y
    giskard_wrapper.set_json_goal('InsertCylinder',
                              cylinder_name=cylinder_name,
                              cylinder_height=cylinder_height,
                              hole_point=hole_point)
    giskard_wrapper.allow_all_collisions()
    giskard_wrapper.plan_and_execute()
    giskard_wrapper.detach_group(cylinder_name)

### Demo: Place the cylinder to block of the same color

In [9]:
# Define 4 different colors
COLORS = [
    ColorRGBA(0.259, 0.522, 0.957, 1.0), # Blue
    ColorRGBA(0.984, 0.737, 0.02, 1.0), # Green
    ColorRGBA(0.204, 0.659, 0.325, 1.0), # Yellow
    ColorRGBA(0.918, 0.263, 0.208, 1.0), # Red
]

# Initialize demo scenario
demo = TracyDemo(COLORS)
demo.setup_color_blocks()

In [10]:
# Generate a random sample of colors
shuffled_color = random.choices(COLORS, k=10)
# shuffled_color = random.sample(COLORS, len(COLORS))

In [11]:
# Spawn cylinders and place to target locations
for color in shuffled_color:
    # randomly choose left or right hand
    hand = random.choice(['r', 'l'])
    # Spawn a cylinder in the choosen hand
    cylinder_name = spawn_cylinder(color=(color.r, color.g, color.b, color.a), hand=hand)
    # Get the target location
    target_pos = demo.get_color_pos(color=color, hand=hand)
    # execute the motion
    insert_cylider_to(target_pos.x, target_pos.y, cylinder_name)
    reset_to_default_pose()